# Predict Credit card Approval
Made by
- Yannick K. Hartington

Our task her is to build a machine learning model to predict if an applicant is 'good' or 'bad' client, were the definition of 'good' or 'bad' is not given. We should also use some techique, such as vintage analysis to construct our label. Lastly, unbalance data problem is a big problem in this task, which we therefore also need to handle. 

### Import Libraries

In [ ]:
import pandas as pd
import os

## Inspection of Dataset
We will start out with inspecting the data to see what we are working with and if it is proper balanced.

In [ ]:
dataset_path = r'/mnt/c/Datasets/Credit Card Approval' # WSL
application_record_path = os.path.join(dataset_path, 'application_record')
credit_record_path = os.path.join(dataset_path, 'credit_record')

## Algorithms
We will try out different kinds of algorithms to see, which one performs the best:
1. Logistic Regression
1. Decision Tree
1. Random Forest
1. Neural Network
